In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
import shutil
import time
import datetime
# load libraries for pair generation
from concurrent.futures import ThreadPoolExecutor
import random
import multiprocessing as mp
import xarray as xr
import os
import sys
import glob
from exactextract import exact_extract
import warnings

In [2]:
import json


def read_key_value_file(filepath):
    with open(filepath, "r") as file:
        data = json.load(file)
    
    key_value_list = [(key, value) for key, value in data.items()]
    return key_value_list

In [3]:
file_path = "/media/volume/NeuralHydrology/Test_Quinn_Data/all_pairs.txt"
key_value_pairs = read_key_value_file(file_path)

print(len(key_value_pairs))

10023


In [4]:
comids = key_value_pairs

In [5]:
comids = [3441628, 3438398]

In [ ]:
def process_chrtout_file2(filename, id_list, value_col="streamflow"):
    with xr.open_dataset(filename) as ds:
        # Select only the desired feature_ids and the streamflow variable
        subset = ds[[value_col]].sel(feature_id=id_list)
        return subset

# Process multiple CHRTOUT files and concatenate on feature_id
def get_q(qlat_files, id_list, value_col="streamflow"):
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(
            process_chrtout_file2,
            [(filename, id_list, value_col) for filename in qlat_files]
        )
    
    # Concatenate along time dimension first (since each file likely represents a timestep)
    combined = xr.concat(results, dim="time")

    return combined

In [ ]:
q_dataset = get_q(glob.glob("/mnt/IMP_DATA/CHRTOUT/*/*.CHRTOUT_DOMAIN1", recursive=True), comids)

In [28]:
target_directory = '/media/volume/NeuralHydrology/Test_Quinn_Data/Channel_Routing/2008/'

In [29]:
feature_id_list = q_dataset['feature_id'].values
for i in range(len(feature_id_list)):
    file_path = target_directory + str(feature_id_list[i]) + '.nc'
    q_dataset.isel(feature_id = i).to_netcdf(file_path)

3438398


In [22]:
print(q_dataset.isel(feature_id = 1))

<xarray.Dataset> Size: 208B
Dimensions:     (time: 24)
Coordinates:
    feature_id  int64 8B 3438398
    latitude    float32 4B 32.01
    longitude   float32 4B -85.27
Dimensions without coordinates: time
Data variables:
    streamflow  (time) float64 192B 0.01 0.01 0.01 0.01 ... 0.01 0.01 0.01 0.01
Attributes: (12/20)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.3.0-alpha1
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2008-01-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2008-01-01_23:00:00
    ...                         ...
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...
    NCO:                        netCDF Operators version 5.1.4 (Homepage = ht...
    history:                    Thu S

In [ ]:
target_filepath = '/media/volume/NeuralHydrology/Test_Quinn_Data/Channel_Routing/1991/'

<xarray.Dataset> Size: 416B
Dimensions:     (time: 24, feature_id: 2)
Coordinates:
  * feature_id  (feature_id) int64 16B 3441628 3438398
    latitude    (feature_id) float32 8B 32.0 32.01
    longitude   (feature_id) float32 8B -85.27 -85.27
Dimensions without coordinates: time
Data variables:
    streamflow  (time, feature_id) float64 384B 0.0 0.01 0.0 ... 0.01 0.0 0.01
Attributes: (12/20)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.3.0-alpha1
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2008-01-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2008-01-01_23:00:00
    ...                         ...
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...
    NCO:                        netCDF Opera

In [9]:
dum_dataset = xr.open_dataset("/mnt/IMP_DATA/CHRTOUT/2008/200811091700.CHRTOUT_DOMAIN1")

In [10]:
dum_dataset

<xarray.Dataset> Size: 200MB
Dimensions:         (feature_id: 2776734, time: 1, reference_time: 1)
Coordinates:
  * time            (time) datetime64[ns] 8B 2008-11-09T17:00:00
  * reference_time  (reference_time) datetime64[ns] 8B 2008-10-01
  * feature_id      (feature_id) int64 22MB 101 179 ... 1180001803 1180001804
    latitude        (feature_id) float32 11MB ...
    longitude       (feature_id) float32 11MB ...
Data variables:
    qBtmVertRunoff  (feature_id) float64 22MB ...
    qBucket         (feature_id) float64 22MB ...
    qSfcLatRunoff   (feature_id) float64 22MB ...
    q_lateral       (feature_id) float64 22MB ...
    streamflow      (feature_id) float64 22MB ...
    velocity        (feature_id) float64 22MB ...
    crs             |S1 1B ...
    order           (feature_id) int32 11MB ...
    elevation       (feature_id) float32 11MB ...
Attributes: (12/20)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.3.0-alpha1
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2008-10-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2008-11-09_17:00:00
    ...                         ...
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...
    NCO:                        netCDF Operators version 5.1.4 (Homepage = ht...
    history:                    Thu Sep 28 09:27:23 2023: ncatted -O -a missi...